#Importo librerie

In [1]:
import pandas as pd
import numpy as np
import concurrent.futures
import time
from requests.exceptions import ReadTimeout

In [2]:
!pip install -U -q PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Autenticazione Spotify API

In [ ]:
#!pip uninstall spotipy

In [ ]:
!pip install spotipy

In [4]:
# autenticazione Spotify API con spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
auth_manager = SpotifyClientCredentials(client_id='caf57b996b464996bff50ab59186f265', client_secret='0bfcdbff8015426cae855b56b692f69b')
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10)

#Importo dataset

In [5]:
# autenticazione google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# importo terza parte del dataset --> 7087 datapoints

drive.CreateFile({'id':'1YAAvPUaPeBIddkuVzWgw7fhPPcT2uJTY'}).GetContentFile('billboard_dataset_unique.csv')
df_billboard = pd.read_csv("billboard_dataset_unique.csv").drop('Unnamed: 0',axis=1).iloc[14174:21261]

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)


# df_billboard.iloc[:7087]

# df_billboard.iloc[7087:14174]

# df_billboard.iloc[14174:21261]

# df_billboard.iloc[21261:]

In [ ]:
df_billboard.head()

,title,artist,weeks
14174,Anywhere With You,Rubber Rodeo,5
14175,Bullish,Herb Alpert's Tijuana Brass,2
14176,Legs,ZZ Top,19
14177,"Mama, Weer All Crazee Now",Quiet Riot,12
14178,She's Mine,Steve Perry,13


In [ ]:
df_billboard.shape

(7087, 3)

#Definizione funzioni

In [7]:
def print_exec_time(start):
  print("Esecuzione completata in %.4f secondi" % (time.perf_counter()-start))

In [8]:
# funzione che effettua ricerca con Spotify API considerando i casi in cui nel campo 'artist' siano presenti più artisti (featuring)

def search_fix(artist, title):
  artist_separators = ['%%%', ' Featuring', ' featuring', ' feat.', ' Feat.', ' feat', ' Feat', ' &', ' x', ' X', ' with', ' With', ', ', '/', ' duet', ' Duet', '+', ' and']
  title_separators = ['%%%', ' (']
  title_fix = ["%%%", "'s", "'"]

  id = None

  for x in artist_separators:
    for y in title_separators:
      for z in title_fix:
        try:
          id = sp.search(q='artist:'+artist.split(x)[0]+' track:'+title.split(y)[0].replace(z, ''), type='track', limit=1)['tracks']['items'][0]['id']
        except IndexError:
          pass
        if(id != None):
          break
      if(id != None):
        break
    if(id != None):
      break

  return id

In [9]:
# funzione che prendendo una singola riga del Billboard dataset restituisce una lista con id, artista e titolo
# --> in caso di errore l'id viene impostato a None

def get_id(row):

  artist = row[1]
  title = row[0]

  print("fetching id for %s by %s ..." % (title, artist))

  try:
    try:
      id = sp.search(q='artist:'+artist+' track:'+title, type='track', limit=1)['tracks']['items'][0]['id']
    except IndexError:
      id = search_fix(artist, title)
  except ReadTimeout:
    id = None
      
  if(id == None):
    print('--> [error] %s by %s' % (title, artist))

  return [id, artist, title]

In [10]:
# funzione che, preso un id, restituisce un array con le features (audio e non) della traccia corrispondente

def get_features(id):
  print("fetching features for id: %s" % id)

  # audio features
  danceability = []
  energy = []
  key = []
  loudness =[]
  mode = []
  speechiness = []
  acousticness = []
  instrumentalness = []
  liveness = []
  valence = []
  tempo = []
  duration_ms = []

  audio_features_array = [danceability, energy, key, loudness, mode, speechiness,
      acousticness, instrumentalness, liveness, valence, tempo, duration_ms]

  # altre features
  release_date = []
  explicit = []

  release_date.append(sp.track(id)['album']['release_date'])
  explicit.append(sp.track(id)['explicit'])

  audio_features = sp.audio_features(id)[0]

  try:
    # rimuovo campi non necessari
    to_remove = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature']
    for rmv in to_remove:
        audio_features.pop(rmv)

    for i, feature in enumerate(audio_features.keys()):
        audio_features_array[i].append(audio_features[feature])

  except AttributeError:
    print("--> [error] id = %s" % id)

    for i in range(12):
        audio_features_array[i].append(None)


  audio_features_array.append(release_date)
  audio_features_array.append(explicit) 

  return audio_features_array

#Integrazione dataset

##Recupero gli id del Billboard dataset

In [ ]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_id, df_billboard.values.tolist())
  
  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

Streaming output truncated to the last 5000 lines.
fetching id for I Go Crazy by Paul Davis ...
fetching id for Mama Let Him Play by Doucette ...
fetching id for More Than A Woman by Tavares ...
fetching id for Africanism/Gimme Some Lovin' by Kongas ...
fetching id for Make You Feel Love Again by Wet Willie ...
fetching id for Fooling Yourself (The Angry Young Man) by Styx ...
fetching id for Ain't Gonna Eat Out My Heart Anymore by Angel ...
fetching id for (Love Is) Thicker Than Water by Andy Gibb ...
fetching id for Ebony Eyes by Bob Welch ...
fetching id for Thank You For Being A Friend by Andrew Gold ...
fetching id for Always And Forever by Heatwave ...
fetching id for Emotion by Samantha Sang ...
fetching id for You'll Love Again by Hotel ...
fetching id for Do You Wanna Dance by Ramones ...
fetching id for Yank Me, Crank Me by Ted Nugent ...
fetching id for Lady Love by Lou Rawls ...
fetching id for Just The Way You Are by Billy Joel ...
fetching id for Music, Harmony And Rhythm

In [ ]:
# creo backup del billboard dataset
df_billboard_bak = df_billboard.copy()

In [ ]:
# inserisco gli id ottenuti in una nuova colonna nel df_billboard
ids = np.array(output)[:,0]
df_billboard.insert(0, 'id', ids)

In [ ]:
# calcolo percentuale di canzoni trovate
found_id = df_billboard.id.count()
x = (found_id / df_billboard.title.count()) * 100
print("Found ids = %d%%" % x)

Found ids = 85%


In [ ]:
# esporto su google drive
from google.colab import drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+ids_2.csv')

Mounted at /content/drive


##Recupero audio features del Billboard dataset

In [11]:
# reimporto dataset billboard (con ids) + dataset principale

drive.CreateFile({'id':'1Y_6NTQjlw3nqffC200Kv79lK9UXJzsA5'}).GetContentFile('billboard+ids_2.csv') 
df_billboard = pd.read_csv("billboard+ids_2.csv").drop('Unnamed: 0',axis=1)

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)

In [12]:
# elimino valori nulli (= id non trovati)
df_billboard = df_billboard.dropna()

# creo lista con gli id del dataset billboard
ids = list(df_billboard.id.array)

In [13]:
# creo lista degli id che non sono presenti nel dataset principale
time_0 = time.perf_counter()
ids_new = [id for id in ids if id not in list(df_songs.id.array)]
print_exec_time(time_0)

Esecuzione completata in 1587.2136 secondi


In [14]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_features, ids_new)

  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

fetching features for id: 2jlcUz84FigsD2IklXSsGI
fetching features for id: 3taPq5NfPwyHv9sTllex4I
fetching features for id: 3zY0g3bPtYJmIoO9shlTiU
fetching features for id: 1rLQBnmGJnZALKuqYm9fJY
fetching features for id: 03xU1bLqUkwo4ae3flK5Di
fetching features for id: 5EuHpH5tpXkXpGHt8Zlafa
fetching features for id: 2Zem6ufFAJBiXpimCFaDPd
fetching features for id: 1APgqlGcOETzgrquIUWlVh
fetching features for id: 6NW5mX8pIyF4BeTNaw7pEZ
fetching features for id: 70GAjPsFUwdyokfXlJ9eWG
fetching features for id: 715sEYWkafd4xvl87dwZgu
fetching features for id: 6xRvKADwRqGUUD5R2LhefZ
fetching features for id: 7o9uu2GDtVDr9nsR7ZRN73
fetching features for id: 3nW0m0mAawgip9cbXAEfnE
fetching features for id: 1spr3ff7dxssldzMZmoITf
fetching features for id: 2OOM7hg35KaqYbDwTmbbk5
fetching features for id: 0RdUX4WE0fO30VnlUbDVL6
fetching features for id: 3Ht0kBuOH7z17OzOOghWKf
fetching features for id: 6m88CdvbaNrCR10JXOW5JQ
fetching features for id: 5yqIBcH5TWKZ6pgPlHXnpW
fetching features fo

#Inserisco audio features nel dataset Billboard

In [15]:
num_datapoints = np.array(output).shape[0]

output = np.array(output).reshape((num_datapoints,14))

In [16]:
# creo backup df_billboard
df_billboard_bak = df_billboard.copy()

# filtro dataset billboard tenendo solo id nell'array 'ids_new', quindi quelli che non sono presenti nel dataset principale
df_billboard = df_billboard[df_billboard.id.isin(ids_new)]

In [17]:
to_insert = ['danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',  
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms',
    'release_date',
    'explicit']

for i, col in enumerate(output.T):
    df_billboard.insert(4, to_insert[i], col)

In [18]:
# converto colonna 'release_date' in tipo datetime
df_billboard.release_date = pd.to_datetime(df_billboard.release_date,format="%Y-%m-%d",exact=False)

In [19]:
# inserisco colonna 'year'
year = df_billboard['release_date'].apply(lambda x: int(x.year))
df_billboard.insert(6, 'year', year)

In [20]:
# inserisco colonna 'popularity' --> nb: inizializzo a 0 perchè verrà rimossa
df_billboard.insert(17, 'popularity', np.zeros(df_billboard.shape[0]))

In [21]:
# inserisco colonna 'hit'
hit = np.ones(df_billboard.shape[0])
df_billboard.insert(3, 'hit', hit)
df_billboard.hit = df_billboard.hit.apply(int)

In [22]:
df_billboard.head()

,id,title,artist,hit,weeks,explicit,release_date,year,duration_ms,tempo,valence,liveness,instrumentalness,acousticness,speechiness,mode,loudness,key,popularity,energy,danceability
0,2jlcUz84FigsD2IklXSsGI,Anywhere With You,Rubber Rodeo,1,5,False,1984-01-01,1984,279533,164.568,0.75,0.153,0.523,0.000236,0.0392,1,-7.892,0,0.0,0.834,0.56
5,3taPq5NfPwyHv9sTllex4I,Satisfy Me,Billy Satellite,1,6,False,1984-01-01,1984,221827,126.409,0.574,0.224,0,0.0306,0.075,1,-4.542,5,0.0,0.897,0.574
6,3zY0g3bPtYJmIoO9shlTiU,The Only Flame In Town,Elvis Costello & The Atrractions,1,9,False,1984-01-01,1984,241560,123.104,0.914,0.116,9.27e-06,0.0996,0.0315,0,-9.245,6,0.0,0.705,0.672
10,1rLQBnmGJnZALKuqYm9fJY,Can't Wait All Night,Juice Newton,1,6,False,1984-07-13,1984,239267,101.095,0.779,0.106,2.59e-05,0.202,0.03,1,-7.076,6,0.0,0.775,0.622
12,03xU1bLqUkwo4ae3flK5Di,I'm Free (Heaven Helps The Man),Kenny Loggins,1,14,False,2014-07-17,2014,226267,160.253,0.814,0.27,0,0.0901,0.039,1,-6.308,8,0.0,0.807,0.585


#Esporto

In [23]:
# esporto in google drive
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+features_2.csv')

Mounted at /content/drive
